# Generate a stone like geometry

Idea here is to generate a set of random points, pick some in the middle that are inside an ellipsoid and compute the surface of the Voronoi tesselation of the points inside with the points outside.